In [1]:
import networkx as nx
from networkx.algorithms import community 
import pandas as pd


In [61]:
df = pd.read_csv('clickstream-enwiki-2020-01.tsv',nrows=2000,sep='\t',header=None)

In [62]:
df.columns = ['From','To','RelationType','Count']

In [63]:
df = df[df['RelationType']=='link']

In [64]:
df

,From,To,RelationType,Count
0,Eddie_Albert,The_Dude_Goes_West,link,17
2,Gale_Storm,The_Dude_Goes_West,link,15
5,Ascoli_Calcio_1898_F.C.,Gianluca_Scamacca,link,87
7,2019–20_Coppa_Italia,Gianluca_Scamacca,link,333
8,2018_UEFA_European_Under-19_Championship,Gianluca_Scamacca,link,23
...,...,...,...,...
1995,Can't_You_Hear_Me_Knocking,Bobby_Keys,link,155
1996,Second_Helping,Bobby_Keys,link,23
1997,Photograph_(Ringo_Starr_song),Bobby_Keys,link,39
1998,Steel_Wheels/Urban_Jungle_Tour,Bobby_Keys,link,11


In [65]:
nodes_from = df['From'].values
nodes_to = df['To'].values
#nodes_from += nodes_to

In [66]:
nodes_from

array(['Eddie_Albert', 'Gale_Storm', 'Ascoli_Calcio_1898_F.C.', ...,
       'Photograph_(Ringo_Starr_song)', 'Steel_Wheels/Urban_Jungle_Tour',
       'The_New_Barbarians_(band)'], dtype=object)

In [ ]:
np.concatenate((nodes_from, nodes_to))

In [34]:
edges = []
for r in df.values:
    edge = (r[0],r[1])
    edges.append(edge)


In [37]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [39]:
density = nx.density(G)
density

0.00034049776198952333

In [41]:
triadic_closure = nx.transitivity(G)
triadic_closure

0

In [43]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

In [45]:
degree_dict

{'Eddie_AlbertThe_Dude_Goes_West': 0,
 'Gale_StormThe_Dude_Goes_West': 0,
 'Ascoli_Calcio_1898_F.C.Gianluca_Scamacca': 0,
 '2019–20_Coppa_ItaliaGianluca_Scamacca': 0,
 '2018_UEFA_European_Under-19_ChampionshipGianluca_Scamacca': 0,
 '2019_FIFA_U-20_World_Cup_squadsGianluca_Scamacca': 0,
 'U.S._Sassuolo_CalcioGianluca_Scamacca': 0,
 '2021_UEFA_European_Under-21_Championship_qualification_Group_1Gianluca_Scamacca': 0,
 'Italy_national_under-21_football_teamGianluca_Scamacca': 0,
 'Thomas_MertonMahanambrata_Brahmachari': 0,
 'Prabhu_JagadbandhuMahanambrata_Brahmachari': 0,
 'Barisal_DistrictMahanambrata_Brahmachari': 0,
 'Birmingham_(disambiguation)Birmingham,_Iowa': 0,
 'Cabinet_of_PakistanUsman_Dar': 0,
 'Imran_Khan_ministryUsman_Dar': 0,
 'Khawaja_Muhammad_AsifUsman_Dar': 0,
 'Halas_Hall1963_NFL_Championship_Game': 0,
 'Super_Bowl_XX1963_NFL_Championship_Game': 0,
 'George_Halas1963_NFL_Championship_Game': 0,
 'Richie_Petitbon1963_NFL_Championship_Game': 0,
 '1963_NFL_season1963_NFL_Ch